In [68]:
h2o.shutdown(prompt=False)

[WARNING] in <ipython-input-68-02ff3806b010> line 1:
    >>> h2o.shutdown(prompt=False)
        ^^^^ Deprecated, use ``h2o.cluster().shutdown()``.
Exception ignored in:<bound method ExprNode.__del__ of <Expr(h2o.runif <Expr(as.numeric <Expr()#data.hex>)#py_21_sid_b4c3; scalar> -1)#py_24_sid_b4c3; scalar>>
Traceback (most recent call last):
  File "/home/dfernandez/.local/lib/python3.6/site-packages/h2o/expr.py", line 187, in __del__
ExprNode.rapids("(rm {})".format(self._cache._id))
  File "/home/dfernandez/.local/lib/python3.6/site-packages/h2o/expr.py", line 241, in rapids
return h2o.api("POST /99/Rapids", data={"ast": expr, "session_id": h2o.connection().session_id})
  File "/home/dfernandez/.local/lib/python3.6/site-packages/h2o/h2o.py", line 104, in api
return h2oconn.request(endpoint, data=data, json=json, filename=filename, save_to=save_to)
  File "/home/dfernandez/.local/lib/python3.6/site-packages/h2o/backend/connection.py", line 434, in request
return self._process_response(r

In [69]:
import h2o
import numpy as np
import pandas as pd

In [70]:
h2o.init(nthreads = -1, max_mem_size = 4)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /home/dfernandez/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpoe061toy
  JVM stdout: /tmp/tmpoe061toy/h2o_dfernandez_started_from_python.out
  JVM stderr: /tmp/tmpoe061toy/h2o_dfernandez_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,04 secs
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.11
H2O cluster version age:,3 days
H2O cluster name:,H2O_from_python_dfernandez_3s3zuc
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [71]:
df = h2o.import_file('Input/data.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [72]:
df.head()

carat,cut,color,clarity,depth,table,x,y,z,price
2.26,Ideal,G,SI2,61.9,57,8.44,8.36,5.2,12831
2.43,Very Good,H,SI2,63.2,57,8.56,8.5,5.39,16170
0.8,Premium,F,SI2,61,57,6.03,6.01,3.67,2797
0.4,Ideal,F,I1,63.3,60,4.68,4.64,2.95,630
0.31,Ideal,G,VS2,61.6,55,4.39,4.37,2.7,698
0.53,Very Good,F,SI2,60.5,60,5.19,5.23,3.15,1132
0.7,Premium,D,SI2,60.8,61,5.69,5.64,3.45,1987
1.13,Good,F,SI2,64,58,6.52,6.58,4.19,3952
0.35,Ideal,E,SI1,60.9,55,4.53,4.57,2.77,614
1.01,Fair,H,VS1,61.4,66,6.34,6.24,3.87,5062


In [73]:
df.types

{'carat': 'real',
 'cut': 'enum',
 'color': 'enum',
 'clarity': 'enum',
 'depth': 'real',
 'table': 'real',
 'x': 'real',
 'y': 'real',
 'z': 'real',
 'price': 'int'}

In [74]:
df = df.asnumeric()

In [75]:
df.types

{'carat': 'real',
 'cut': 'real',
 'color': 'real',
 'clarity': 'real',
 'depth': 'real',
 'table': 'real',
 'x': 'real',
 'y': 'real',
 'z': 'real',
 'price': 'real'}

In [76]:
df.head()

carat,cut,color,clarity,depth,table,x,y,z,price
2.26,2,3,3,61.9,57,8.44,8.36,5.2,12831
2.43,4,4,3,63.2,57,8.56,8.5,5.39,16170
0.8,3,2,3,61,57,6.03,6.01,3.67,2797
0.4,2,2,0,63.3,60,4.68,4.64,2.95,630
0.31,2,3,5,61.6,55,4.39,4.37,2.7,698
0.53,4,2,3,60.5,60,5.19,5.23,3.15,1132
0.7,3,0,3,60.8,61,5.69,5.64,3.45,1987
1.13,1,2,3,64,58,6.52,6.58,4.19,3952
0.35,2,1,2,60.9,55,4.53,4.57,2.77,614
1.01,0,4,4,61.4,66,6.34,6.24,3.87,5062


In [77]:
y_columns = "price"
x_columns = ["carat","cut","color", "clarity","depth","table","x","y","z"]

In [78]:
train, test=df.split_frame(ratios = [.8])
X_train=train[x_columns]
y_train=train[y_columns]
X_test=test[x_columns]
y_test=test[y_columns]

In [79]:
from h2o.estimators.random_forest import H2ORandomForestEstimator as rf

In [80]:
rf_fit1 = rf(model_id='rf_fit1', seed=1)
rf_fit1.train(x=x_columns, y=y_columns, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [81]:
rf_fit1.model_performance

Model Details
H2ORandomForestEstimator:Distributed Random Forest
Model Key:rf_fit1


Model Summary:


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 381275.8370752573
RMSE: 617.4753736589479
MAE: 307.3862521405956
RMSLE: 0.10500320810189523
Mean Residual Deviance: 381275.8370752573

Scoring History:

Variable Importances:


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,9518263.0,20.0,20.0,20.0,14164.0,15798.0,15175.06


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2019-12-09 15:10:04,0.077 sec,0.0,NaN,NaN,NaN
1,,2019-12-09 15:10:05,1.461 sec,1.0,1043.799599,480.283698,1.089518e+06
2,,2019-12-09 15:10:06,1.916 sec,2.0,938.051689,447.399976,8.799410e+05
3,,2019-12-09 15:10:06,2.199 sec,3.0,884.106172,425.924906,7.816437e+05
4,,2019-12-09 15:10:06,2.509 sec,4.0,865.917696,415.910167,7.498135e+05
5,,2019-12-09 15:10:07,2.822 sec,5.0,822.993188,399.692856,6.773178e+05
6,,2019-12-09 15:10:07,3.098 sec,6.0,823.070414,398.418825,6.774449e+05
7,,2019-12-09 15:10:07,3.327 sec,7.0,803.372916,391.678259,6.454080e+05
8,,2019-12-09 15:10:07,3.542 sec,8.0,771.268590,380.657853,5.948552e+05
9,,2019-12-09 15:10:07,3.747 sec,9.0,754.843345,374.280352,5.697885e+05


,variable,relative_importance,scaled_importance,percentage
0,y,7.602560e+12,1.000000,0.356505
1,carat,4.779768e+12,0.628705,0.224137
2,x,4.355203e+12,0.572860,0.204228
3,z,2.938757e+12,0.386548,0.137807
4,clarity,8.878097e+11,0.116778,0.041632
5,color,5.203172e+11,0.068440,0.024399
6,depth,1.010061e+11,0.013286,0.004736
7,table,8.794023e+10,0.011567,0.004124
8,cut,5.187732e+10,0.006824,0.002433


<bound method ModelBase.model_performance of >

In [82]:
from h2o.automl import H2OAutoML
aml_ti = H2OAutoML(max_runtime_secs= 180,max_models= 15, seed= 1, nfolds=0)
aml_ti.train(x = x_columns, y = y_columns, training_frame = train, validation_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [83]:
lb_ti = aml_ti.leaderboard
lb_ti

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_3_AutoML_20191209_151417,290980,539.426,290980,278.082,0.0987834
GBM_1_AutoML_20191209_151417,291358,539.776,291358,284.019,0.100834
GBM_4_AutoML_20191209_151417,291423,539.836,291423,276.01,0.092148
GBM_2_AutoML_20191209_151417,297531,545.464,297531,283.799,0.101326
XGBoost_3_AutoML_20191209_151417,302785,550.259,302785,288.095,0.104966
XGBoost_1_AutoML_20191209_151417,302786,550.26,302786,274.924,0.0920021
XGBoost_2_AutoML_20191209_151417,310489,557.215,310489,275.503,0.0889387
GBM_5_AutoML_20191209_151417,331264,575.556,331264,284.644,0.0983653
XRT_1_AutoML_20191209_151417,355688,596.396,355688,302.069,0.101052
DRF_1_AutoML_20191209_151417,363766,603.13,363766,303.066,0.102634
